# python-MIP

In [ ]:
!pip install mip

In [ ]:
from mip import *
from sys import stdout

## knapsack

In [ ]:
p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

m += xsum(w[i] * x[i] for i in I) <= c

m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [0, 3]


## n-queens

In [ ]:
# number of queens
n = 10

queens = Model()

x = [[queens.add_var('x({},{})'.format(i, j), var_type=BINARY)
      for j in range(n)] for i in range(n)]

# one per row
for i in range(n):
    queens += xsum(x[i][j] for j in range(n)) == 1, 'row({})'.format(i)

# one per column
for j in range(n):
    queens += xsum(x[i][j] for i in range(n)) == 1, 'col({})'.format(j)

# diagonal \
for p, k in enumerate(range(2 - n, n - 2 + 1)):
    queens += xsum(x[i][i - k] for i in range(n)
                   if 0 <= i - k < n) <= 1, 'diag1({})'.format(p)

# diagonal /
for p, k in enumerate(range(3, n + n)):
    queens += xsum(x[i][k - i] for i in range(n)
                   if 0 <= k - i < n) <= 1, 'diag2({})'.format(p)

queens.optimize()

if queens.num_solutions:
    stdout.write('\n')
    for i, v in enumerate(queens.vars):
        stdout.write('O ' if v.x >= 0.99 else '. ')
        if i % n == n-1:
            stdout.write('\n')


. . . . . . . O . . 
. . O . . . . . . . 
. . . . O . . . . . 
. . . . . . . . . O 
. O . . . . . . . . 
. . . . . . . . O . 
. . . . . O . . . . 
. . . O . . . . . . 
. . . . . . O . . . 
O . . . . . . . . . 


## sport tournament

In [ ]:
n = 4

srrt = Model()

x = [[[[srrt.add_var('x({},{},{},{})'.format(i, j, p, w), var_type=BINARY)
     for w in range(n-1)] for p in range(int(n/2))] for j in range(n)] for i in range(n)]


# one per row
for i in range(n):
    for p in range(int(n/2)):
        for w in range(n-1):
            srrt += xsum(x[i][j][p][w] for j in range(n)) == 1, 'row({})'.format(i)

# one per column
for j in range(n):
    for p in range(int(n/2)):
        for w in range(n-1):
            srrt += xsum(x[i][j][p][w] for i in range(n)) == 1, 'col({})'.format(j)

# srrt.objective = minimize(xsum(x[i][j][p][w] for w in range(n-1) for p in range(int(n/2)) for j in range(n) for i in range(n)))

status = srrt.optimize(max_seconds=300)
print(status)

print(srrt.num_solutions)

print(srrt.vars)
for i, v in enumerate(srrt.vars):
    stdout.write('O ' if v.x >= 0.99 else '. ')
    if i % n == n-1:
        stdout.write('\n')
    if i % n**2 == (n**2-1):
        stdout.write('\n')

OptimizationStatus.OPTIMAL
1
. . . . 
. . . . 
. . . O 
. . . . 

. . O O 
O O O . 
. . . . 
. . O O 

. . . . 
. . O O 
O O . . 
. . . . 

. . . . 
. . . . 
O O O . 
O O . . 

. . . . 
. . . O 
O O O O 
O O . . 

. . . . 
. . . . 
. . . . 
. . . . 



# PuLP

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 53.3 MB/s eta 0:00:00


In [ ]:
from pulp import *

## sudoku

In [ ]:
VALS = ROWS = COLS = range(1, 10)
Boxes = [
    [(3 * i + k + 1, 3 * j + l + 1) for k in range(3) for l in range(3)]
    for i in range(3)
    for j in range(3)
]
prob = LpProblem("Sudoku Problem")
choices = LpVariable.dicts("Choice", (VALS, ROWS, COLS), cat="Binary")
for r in ROWS:
    for c in COLS:
        prob += lpSum([choices[v][r][c] for v in VALS]) == 1

# The row, column and box constraints are added for each value
for v in VALS:
    for r in ROWS:
        prob += lpSum([choices[v][r][c] for c in COLS]) == 1

    for c in COLS:
        prob += lpSum([choices[v][r][c] for r in ROWS]) == 1

    for b in Boxes:
        prob += lpSum([choices[v][r][c] for (r, c) in b]) == 1

# The starting numbers are entered as constraints
input_data = [
    (5, 1, 1),
    (6, 2, 1),
    (8, 4, 1),
    (4, 5, 1),
    (7, 6, 1),
    (3, 1, 2),
    (9, 3, 2),
    (6, 7, 2),
    (8, 3, 3),
    (1, 2, 4),
    (8, 5, 4),
    (4, 8, 4),
    (7, 1, 5),
    (9, 2, 5),
    (6, 4, 5),
    (2, 6, 5),
    (1, 8, 5),
    (8, 9, 5),
    (5, 2, 6),
    (3, 5, 6),
    (9, 8, 6),
    (2, 7, 7),
    (6, 3, 8),
    (8, 7, 8),
    (7, 9, 8),
    (3, 4, 9),
    (1, 5, 9),
    (6, 6, 9),
    (5, 8, 9),
]

for v, r, c in input_data:
    prob += choices[v][r][c] == 1

# The problem data is written to an .lp file
prob.writeLP("Sudoku.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# A file called sudokuout.txt is created/overwritten for writing to
sudokuout = open("sudokuout.txt", "w")

# The solution is written to the sudokuout.txt file
for r in ROWS:
    if r in [1, 4, 7]:
        sudokuout.write("+-------+-------+-------+\n")
    for c in COLS:
        for v in VALS:
            if value(choices[v][r][c]) == 1:
                if c in [1, 4, 7]:
                    sudokuout.write("| ")
                sudokuout.write(str(v) + " ")
                if c == 9:
                    sudokuout.write("|\n")
sudokuout.write("+-------+-------+-------+")
sudokuout.close()

Status: Optimal
Status: Optimal
Status: Infeasible


In [ ]:
sudokuout = open("sudokuout.txt", "w")
while True:
    prob.solve()
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[prob.status])
    # The solution is printed if it was deemed "optimal" i.e met the constraints
    if LpStatus[prob.status] == "Optimal":
        # The solution is written to the sudokuout.txt file
        for r in ROWS:
            if r in [1, 4, 7]:
                sudokuout.write("+-------+-------+-------+\n")
            for c in COLS:
                for v in VALS:
                    if value(choices[v][r][c]) == 1:
                        if c in [1, 4, 7]:
                            sudokuout.write("| ")
                        sudokuout.write(str(v) + " ")
                        if c == 9:
                            sudokuout.write("|\n")
        sudokuout.write("+-------+-------+-------+\n\n")
        # The constraint is added that the same solution cannot be returned again
        prob += (
            lpSum(
                [
                    choices[v][r][c]
                    for v in VALS
                    for r in ROWS
                    for c in COLS
                    if value(choices[v][r][c]) == 1
                ]
            )
            <= 80
        )
    # If a new optimal solution cannot be found, we end the program
    else:
        break
sudokuout.close()

Status: Optimal
Status: Infeasible


# gurobi

In [32]:
!pip install gurobipy

In [33]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

## es.

In [ ]:
env = gp.Env()

model = gp.Model(env=env)
x = model.addVar(vtype=GRB.BINARY, name="x")
y = model.addVar(vtype=GRB.BINARY, name="y")
z = model.addVar(vtype=GRB.BINARY, name="z")

model.setObjective(x + y + 2 * z, GRB.MAXIMIZE)
model.addConstr(x + 2 * y + 3 * z <= 4, "c0")

model.Params.TimeLimit = 10.0
model.optimize()

print(f"x = {x.X}")
print(f"y = {y.X}")
print(f"z = {z.X}")

Restricted license - for non-production use only - expires 2026-11-23
Set parameter TimeLimit to value 10
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  10

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x3eaaf627
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance

## sudoku

In [ ]:
n = 8

m = gp.Model("nqueens")

# n-by-n binary variables; x[i, j] decides whether a queen is placed at
# position (i, j)
x = m.addMVar((n, n), vtype=GRB.BINARY, name="x")

# Maximize the number of placed queens
m.setObjective(x.sum(), GRB.MAXIMIZE)

# At most one queen per row; this adds n linear constraints
m.addConstr(x.sum(axis=1) <= 1, name="row")

# At most one queen per column; this adds n linear constraints
m.addConstr(x.sum(axis=0) <= 1, name="col")

for i in range(-n + 1, n):
    # At most one queen on diagonal i
    m.addConstr(x.diagonal(i).sum() <= 1, name=f"diag{i:d}")

    # At most one queen on anti-diagonal i
    m.addConstr(x[:, ::-1].diagonal(i).sum() <= 1, name=f"adiag{i:d}")

# Solve the problem
m.optimize()

print(x.X)
print(f"Queens placed: {m.ObjVal:.0f}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 46 rows, 64 columns and 256 nonzeros
Model fingerprint: 0xf9127510
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 42 rows, 64 columns, 270 nonzeros
Variable types: 0 continuous, 64 integer (64 binary)

Root relaxation: objective 8.000000e+00, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0        

## RR tournament scheduling

In [169]:
n = 8
m = gp.Model("RR Tournament Scheduling")

x = m.addMVar((n-1, int(n/2), n, n), vtype=GRB.BINARY, name="x")

# one match against each team
m.addConstrs((
    gp.quicksum(x[i1, i2, i, i4] for i1 in range(n-1) for i2 in range(int(n/2)) for i4 in range(n)) +
    gp.quicksum(x[i1, i2, i3, i] for i1 in range(n-1) for i2 in range(int(n/2)) for i3 in range(n))
    == n - 1
    for i in range(n)
), name="ij")

# m.addConstrs((x[:, :, i, :].sum() == n-(i+1) for i in range(n)), name="ij")
# m.addConstrs((x[:, :, :, j].sum() == n-(i+1) for j in range(n)), name="ij")
# m.addConstrs((x[:, :, i, :].sum() == n-1 for i in range(n)), name="row")
# m.addConstrs((x[:, :, :, j].sum() == n-1 for j in range(n)), name="col")
# no match against itself
m.addConstrs((x[:, :, i, i] == 0 for i in range(n)), name="self")

# diag symmetry breaking
m.addConstrs((
    gp.quicksum(x[i1, i2, i, j] for i1 in range(n-1) for i2 in range(int(n/2))) +
    gp.quicksum(x[i1, i2, j, i] for i1 in range(n-1) for i2 in range(int(n/2)))
    == 1
    for i in range(n) for j in range(n) if i != j
), name="ij")

# one match per team per week
for w in range(n-1):
    m.addConstrs((x[w, :, i, :].sum() <= 1 for i in range(n)), name="max_w")
    m.addConstrs((x[w, :, :, j].sum() <= 1 for j in range(n)), name="max_w")

# max one match over the periods in a week
for w in range(n-1):
    m.addConstrs((x[w, :, i, j].sum() <= 1 for i in range(n) for j in range(n)), name="max_p")

# max one game per match
m.addConstrs((x[:, :, i, j].sum() <= 1 for i in range(n) for j in range(n)), name="max_ij")

# the num of matches must be n*n-n (just for safety)
# m.addConstr((x[:, :, :, :].sum() == int((n*n-n)/2)), name="tot_matches")


# at most 2 match per period per team
# m.addConstrs((x[:, p, i, :].sum() <= 2 for i in range(n) for p in range(int(n/2))), name="max_ij")
m.addConstrs((
    gp.quicksum(x[w, p, i, j] for w in range(n-1) for j in range(n)) +
    gp.quicksum(x[w, p, j, i] for w in range(n-1) for j in range(n))
    <= 2
    for i in range(n) for p in range(int(n/2))
), name="ij")

m.Params.TimeLimit = 300.0
m.Params.PoolSearchMode = 2
m.Params.PoolSolutions = 5
m.optimize()

Set parameter TimeLimit to value 300
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 5
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  300
PoolSolutions  5
PoolSearchMode  2



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [117]:
print(x.X)

[[[[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]]

  [[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]]]


 [[[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]]

  [[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]]]


 [[[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]]

  [[0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]
   [0. 0. 0. 0.]]]]


In [168]:
m.Params.SolutionNumber = 0
k = x.Xn.copy()
# compress the solution in one array for test purposes
o = np.sum(k, axis=(0, 1))
print(o)

# print of the 3rd constr
num_weeks, num_periods, num_teams, _ = n-1, int(n/2), n, n
for i in range(num_teams):
    print(f"Team {i}:")
    for p in range(num_periods):
        print(f"  Periodo {p}: {np.sum(k[:, p, i, :])+np.sum(k[:, p, :, i])} partite")

[[0. 0. 1. 0. 1. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 1.]
 [1. 1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 1. 1.]
 [1. 0. 1. 1. 1. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 1. 0.]]
Team 0:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite
  Periodo 2: 1.0 partite
  Periodo 3: 2.0 partite
Team 1:
  Periodo 0: 1.0 partite
  Periodo 1: 2.0 partite
  Periodo 2: 2.0 partite
  Periodo 3: 2.0 partite
Team 2:
  Periodo 0: 1.0 partite
  Periodo 1: 2.0 partite
  Periodo 2: 2.0 partite
  Periodo 3: 2.0 partite
Team 3:
  Periodo 0: 1.0 partite
  Periodo 1: 2.0 partite
  Periodo 2: 2.0 partite
  Periodo 3: 2.0 partite
Team 4:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite
  Periodo 2: 2.0 partite
  Periodo 3: 1.0 partite
Team 5:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite
  Periodo 2: 2.0 partite
  Periodo 3: 1.0 partite
Team 6:
  Periodo 0: 1.0 partite
  Periodo 1: 2.0 partite
  Periodo 2: 2.0 partite
  Periodo 3: 2.0 partite
Team 7:
  Periodo 0: 2.0 pa

In [10]:
# print all solutions found
for i in range(m.SolCount):
    m.Params.SolutionNumber = i
    print('Solution number: ', i)
    print(np.reshape(x.Xn, (n-1, int(n/2), n, n)))
    print("")
